In [4]:
import os
import pandas as pd
from pathlib import Path
from newsapi import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer
%matplotlib inline

In [56]:
import pandas as pd

In [5]:
# Load the ticer data
file = Path('./tickers.csv')
tickers_df = pd.read_csv(file)

tickers_df.head()

,Ticker,Name,Description,Price(USD),Start,End,ROI(Pct),Timezone,Duration
0,REP,Augur,Augur is a decentralized prediction market,11.40,2015-08-17 00:00:00,2015-10-01 00:00:00,18.6608,UTC+0,45 days 00:00:00.000000000
1,LSK,Lisk,It is a cryptocurrency and decentralized appli...,0.75,2016-02-22 00:00:00,2016-03-21 00:00:00,9.0516,UTC+0,28 days 00:00:00.000000000
2,DGD,Digix DAO,A Decentralized Autonomous Organization focuse...,12.25,2016-03-30 12:00:00,2016-03-30 12:00:00,2.7809,UTC+0,0 days 00:00:00.000000000
3,WAVES,Waves,Waves helps to make the launching and coordina...,0.78,2016-04-12 13:00:00,2016-05-31 13:00:00,3.1354,UTC+0,49 days 00:00:00.000000000
4,STRAT,Stratis,Stratis was developed to help organisations de...,0.30,2016-06-21 00:00:00,2016-07-26 00:00:00,40.4598,UTC+0,35 days 00:00:00.000000000


In [8]:
tickers_df.dropna(inplace=True)

In [9]:
tickers_dict = tickers_df.set_index('Ticker').to_dict()['Name']

In [27]:
from itertools import islice
#take(5, tickers_dict.items())
list(islice(tickers_dict.items(), 5))

[('REP', 'Augur'),
 ('LSK', 'Lisk'),
 ('DGD', 'Digix DAO'),
 ('WAVES', 'Waves'),
 ('STRAT', 'Stratis')]

In [15]:
api_key = os.getenv("NEWS_API_KEY")

In [16]:
newsapi = NewsApiClient(api_key=api_key)

In [18]:
bit_articles = newsapi.get_everything(q="bitcoin OR BTC", language="en", sort_by='relevancy')

print(f"Total articles about Bitcoin: {bit_articles['totalResults']}")

Total articles about Bitcoin: 4754


In [ ]:
articles_dict = {}
for key, value in tickers_dict.items():
    articles_dict[f"{key}_articles"] = newsapi.get_everything(q=f"{key} OR {value}", language="en", sort_by='relevancy')
    

In [38]:
articles_dict['REP_articles']['articles']

[{'source': {'id': None, 'name': 'Lifehacker.com'},
  'author': 'Josh Ocampo',
  'title': "Here Is Every Democratic Congressmember's Pick for President So Far",
  'description': 'This weekend, New York rep. Alexandria Ocasio-Cortez officially endorsed Vermont Senator Bernie Sanders for president in a rally in Queens, New York to an enthusiastic crowd. And while the rally received some backlash—mainly from the GOP and conservative outl…',
  'url': 'https://lifehacker.com/here-is-every-democratic-congressmembers-pick-for-presi-1839232282',
  'urlToImage': 'https://i.kinja-img.com/gawker-media/image/upload/c_fill,f_auto,fl_progressive,g_center,h_675,pg_1,q_80,w_1200/thx4ly4rzjrenkljwvyt.png',
  'publishedAt': '2019-10-23T16:30:00Z',
  'content': 'This weekend, New York rep. Alexandria Ocasio-Cortez officially endorsed Vermont Senator Bernie Sanders for president in a rally in Queens, New York to an enthusiastic crowd. And while the rally received some backlashmainly from the GOP and conse

In [30]:
list(islice(articles_dict.items(), 1))

[('REP_articles',
  {'status': 'ok',
   'totalResults': 21050,
   'articles': [{'source': {'id': None, 'name': 'Lifehacker.com'},
     'author': 'Josh Ocampo',
     'title': "Here Is Every Democratic Congressmember's Pick for President So Far",
     'description': 'This weekend, New York rep. Alexandria Ocasio-Cortez officially endorsed Vermont Senator Bernie Sanders for president in a rally in Queens, New York to an enthusiastic crowd. And while the rally received some backlash—mainly from the GOP and conservative outl…',
     'url': 'https://lifehacker.com/here-is-every-democratic-congressmembers-pick-for-presi-1839232282',
     'urlToImage': 'https://i.kinja-img.com/gawker-media/image/upload/c_fill,f_auto,fl_progressive,g_center,h_675,pg_1,q_80,w_1200/thx4ly4rzjrenkljwvyt.png',
     'publishedAt': '2019-10-23T16:30:00Z',
     'content': 'This weekend, New York rep. Alexandria Ocasio-Cortez officially endorsed Vermont Senator Bernie Sanders for president in a rally in Queens, New Yor

In [39]:
analyzer = SentimentIntensityAnalyzer()

In [46]:
df_dict = {}
for key in articles_dict:
    sentiments = []
    for article in articles_dict[key]["articles"]:
        try:
            sentiment = analyzer.polarity_scores(article["content"])
      
            sentiments.append({
                "text": article["content"],
                "compound": sentiment["compound"],
                "positive": sentiment["pos"],
                "negative": sentiment["neg"],
                "neutral": sentiment["neu"]
            
            })
        
        except AttributeError:
            pass
      
    df_dict[f"{key}_df"] = pd.DataFrame(sentiments)

In [58]:
df_dict["REP_articles_df"]

,compound,negative,neutral,positive,text
0,0.6369,0.000,0.880,0.120,"This weekend, New York rep. Alexandria Ocasio-..."
1,-0.5095,0.128,0.814,0.058,"SERIOUSLY SCHUMER??, wrote Peter Daou, a forme..."
2,0.8402,0.044,0.760,0.196,"FroYo brand honors veterans with discount, say..."
3,0.0258,0.079,0.839,0.082,Maryland Democratic Rep. Elijah Cummings died ...
4,0.0000,0.000,1.000,0.000,Chat with us in Facebook Messenger. Find out w...
5,0.0000,0.000,1.000,0.000,Chat with us in Facebook Messenger. Find out w...
6,0.0000,0.000,1.000,0.000,Chat with us in Facebook Messenger. Find out w...
7,0.0000,0.000,1.000,0.000,Chat with us in Facebook Messenger. Find out w...
8,0.0000,0.000,1.000,0.000,Chat with us in Facebook Messenger. Find out w...
9,0.0000,0.000,1.000,0.000,Chat with us in Facebook Messenger. Find out w...


In [57]:
df = pd.Dataframe(df_dict["REP_articles_df"].describe().mean())
df

AttributeError: module 'pandas' has no attribute 'Dataframe'

In [59]:
for key, value in df_dict.items():
    file = Path(f"./data_files/{key}.csv")
    value.to_csv(file, index=False)